In [ ]:
%matplotlib inline


# Macondo


In [ ]:
from datetime import datetime, timedelta
import urllib.request as urllib_request
from opendrift.readers import reader_netCDF_CF_generic
from opendrift.models.openoil import OpenOil

This is a highly simplified 2D simulation, for illustration purpose only



In [ ]:
o = OpenOil(loglevel=20)  # Set loglevel to 0 for debug information

This dataset is frequently unavailable, so trying first to "wake" the thredds server:



In [ ]:
for r in ['http://tds0.ifremer.fr', 'http://tds0.ifremer.fr/thredds/dodsC/GLOBCURRENT-L4-CUREUL_15M-ALT_SUM_NRT-V03.0.html']:
    try:
        print('Fetching ' + r)
        request = urllib_request.Request(r)  # Wake up thredds-server, if sleeping
        urllib_request.urlopen(request, timeout=5)
    except:
        pass
try:
    # For this datasource which does not contain standard_name, we impose a variable mapping
    reader_globcurrent = reader_netCDF_CF_generic.Reader(
        'http://tds0.ifremer.fr/thredds/dodsC/GLOBCURRENT-L4-CUREUL_HS-ALT_SUM-V03.0',
        standard_name_mapping={'eastward_eulerian_current_velocity': 'x_sea_water_velocity',
                               'northward_eulerian_current_velocity': 'y_sea_water_velocity'})
except:
    print('Thredds server not available, cannot run example')
    import sys
    sys.exit(0)

reader_oceanwind = reader_netCDF_CF_generic.Reader('https://tds0.ifremer.fr/thredds/dodsC/CERSAT-GLO-CLIM_WIND_L4-OBS_FULL_TIME_SERIE')

# Add readers
o.add_reader([reader_globcurrent, reader_oceanwind])

# Seed some particles
lon = -88.387161; lat = 28.736669  # Macondo location
starttime = datetime(2010, 4, 21, 6, 0, 0)  # 4 hours after explosion
time = [starttime, starttime + timedelta(hours=24*30)]
o.seed_elements(lon, lat, radius=0, number=15000, time=time, oil_type='LIGHT LOUISIANNA SWEET, BP')

# Run model
print(o)
o.run(duration=timedelta(days=30),
      time_step=timedelta(hours=3),
      time_step_output=timedelta(hours=6))

# Print and plot results
print(o)
o.plot(fast=True)
o.plot_oil_budget()
o.animation(fast=True)

<img src="file://gallery/animations/example_macondo_0.gif">

